In [1]:
import ast
import pandas as pd

import src.edge_constructors as edge
import src.clustering as clu
import src.metrics_fr as fr
import src.utils as utils

[Taichi] version 1.7.1, llvm 15.0.7, commit 0f143b2f, osx, python 3.11.8


[I 05/27/24 18:32:23.466 4962603] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=metal


# Thesis Evaluation

## Helper Functions

In [2]:
def load_data(filepath, n=None):
    assert filepath[-4:] == ".csv", "Must be a .csv file"
    data = pd.read_csv(filepath)
    if n:
        data = data.head(n)

    attrs = {
        "titles": data["title"].tolist(),
        "text": data["text"].tolist(),
        "tags": data["tags"].apply(ast.literal_eval).tolist(),
        "ids": data.index.tolist()
    }

    if "simplified_tags" in data.columns:
        attrs["simplified_tags"] = data["simplified_tags"].apply(ast.literal_eval).tolist()

    return attrs

def load_embeddings(dataset_name, model_names):
    embeddings = []
    for name in model_names:
        embeddings.append(utils.load_from_pickle(f"embeddings/{dataset_name}_{name}_n10000.pickle"))
    return embeddings

## Data: interview_prep.csv

In [3]:
data_name = "interview_prep"
data = load_data(f"data/{data_name}.csv")

### Embedding and Similarity Scores

In [4]:
df1 = fr.get_embedding_similarity_metrics_per_dataset("interview_prep", data["tags"],
                                             ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
                                             ["mean"])
df1.to_csv("analysis/metric1_interview.csv")
df1.head(10)

Calculating euclidean similarities: 100%|██████████| 1/1 [00:00<00:00, 2659.67it/s]


,data_source,embedding_model,agg_method,metric_name,metric,between_all_nodes,between_shared_tags
0,interview_prep,minilm,mean,cosine,mean,0.684910,0.706979
1,interview_prep,minilm,mean,cosine,median,0.674775,0.715765
2,interview_prep,minilm,mean,cosine,std_dev,0.069721,0.070315
3,interview_prep,minilm,mean,soft_cosine,mean,0.641387,0.656642
4,interview_prep,minilm,mean,soft_cosine,median,0.644162,0.661419
5,interview_prep,minilm,mean,soft_cosine,std_dev,0.126290,0.134193
6,interview_prep,minilm,mean,euclidean,mean,0.537721,0.561980
7,interview_prep,minilm,mean,euclidean,median,0.521811,0.566406
8,interview_prep,minilm,mean,euclidean,std_dev,0.075639,0.077599
0,interview_prep,mpnet,mean,cosine,mean,0.707133,0.725702


### Cluster Purity, Homogeneity, and Completeness

In [5]:
df2 = fr.compare_cluster_metrics("interview_prep",
                                    ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
                                    ["mean"],
                                    {
                                        "kmeans5": lambda x: clu.kmeans(x),
                                        "kmeans2": lambda x: clu.kmeans(x, 2)
                                    }, data["ids"], data["tags"], k=2)
df2.to_csv("analysis/metric2_interview.csv")
df2.head(10)

,embedding_model,agg_method,clusterer,homogeneity,completeness,tag_concentration_purity,cluster_tag_purity
0,minilm,mean,kmeans5,0.285,0.133,"{'haha': 0.833, 'hehe': 0.667}","{'haha': 0.357, 'hehe': 0.143}"
1,minilm,mean,kmeans2,0.205,0.223,"{'haha': 0.8, 'hehe': 0.75}","{'haha': 0.571, 'hehe': 0.214}"
2,mpnet,mean,kmeans5,0.335,0.139,"{'haha': 0.75, 'hehe': 0.667}","{'haha': 0.214, 'hehe': 0.143}"
3,mpnet,mean,kmeans2,0.084,0.106,"{'haha': 0.727, 'hehe': 0.273}","{'haha': 0.571, 'hehe': 0.214}"
4,nomic,mean,kmeans5,0.430,0.178,"{'haha': 1.0, 'hehe': 0.667}","{'haha': 0.286, 'hehe': 0.143}"
5,nomic,mean,kmeans2,0.084,0.106,"{'haha': 0.727, 'hehe': 0.273}","{'haha': 0.571, 'hehe': 0.214}"
6,bert,mean,kmeans5,0.191,0.086,"{'haha': 0.833, 'hehe': 0.5}","{'haha': 0.357, 'hehe': 0.071}"
7,bert,mean,kmeans2,0.121,0.305,"{'haha': 0.692, 'hehe': 0.308}","{'haha': 0.643, 'hehe': 0.286}"
8,specter,mean,kmeans5,0.544,0.247,"{'haha': 0.75, 'hehe': 0.5}","{'haha': 0.214, 'hehe': 0.214}"
9,specter,mean,kmeans2,0.084,0.106,"{'haha': 0.727, 'hehe': 0.273}","{'haha': 0.571, 'hehe': 0.214}"


### Edge Assignment Evaluation: Tag Connectivity and Degree of Separation

In [6]:
df3 = fr.compare_edge_assignment_metrics("interview_prep",
                                        ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
                                        ["mean"],
                                        {
                                            "kmeans5": lambda x: clu.kmeans(x),
                                            "kmeans2": lambda x: clu.kmeans(x, 2)
                                        },
                                        {
                                            "knn3": lambda sim_mat, ids: edge.knn(sim_mat, ids, k=3),
                                            "knn5": lambda sim_mat, ids: edge.knn(sim_mat, ids, k=5)
                                        }, data["ids"], data["tags"], data["titles"], max_depth=3)
df3.to_csv("analysis/metric3_interview.csv")
df3.head(10)

Calculating euclidean similarities: 100%|██████████| 1/1 [00:00<00:00, 628.93it/s]


graphs/interview_prep_minilm_cosine_mean_knn3_kmeans5


Calculating cosine similarities: 100%|██████████| 1/1 [00:00<00:00, 2841.67it/s]


graphs/interview_prep_minilm_cosine_mean_knn3_kmeans2


Calculating cosine similarities: 100%|██████████| 1/1 [00:00<00:00, 2513.06it/s]


...

graphs/interview_prep_word2vec_euclidean_mean_knn5_kmeans5


Calculating euclidean similarities: 100%|██████████| 1/1 [00:00<00:00, 2065.14it/s]

graphs/interview_prep_word2vec_euclidean_mean_knn5_kmeans2



Calculating euclidean similarities: 100%|██████████| 1/1 [00:00<00:00, 1211.88it/s]


,embedding_model,agg_method,similarity,edge constructor,clusterer,depth,connected_nodes,percentage_connected,degree_of_separation
0,minilm,mean,cosine,knn3,kmeans5,1,2,0.100,1.668210
1,minilm,mean,cosine,knn3,kmeans5,2,3,0.150,1.668210
2,minilm,mean,cosine,knn3,kmeans5,3,11,0.550,1.668210
3,minilm,mean,cosine,knn3,kmeans2,1,2,0.118,1.338812
4,minilm,mean,cosine,knn3,kmeans2,2,7,0.412,1.338812
5,minilm,mean,cosine,knn3,kmeans2,3,4,0.235,1.338812
6,minilm,mean,cosine,knn5,kmeans5,1,1,0.050,1.610567
7,minilm,mean,cosine,knn5,kmeans5,2,4,0.200,1.610567
8,minilm,mean,cosine,knn5,kmeans5,3,13,0.650,1.610567
9,minilm,mean,cosine,knn5,kmeans2,1,2,0.118,1.277447


## Data: Medium (n=10000)

In [7]:
data_name = "medium_1k_tags_simplified"
data = load_data(f"data/{data_name}.csv", n=5000)
ids = data["ids"]
titles = data["titles"]
tags = data["simplified_tags"]

### Embedding and Similarity Scores

In [8]:
# df1 = fr.get_embedding_similarity_metrics_per_dataset("medium1k", tags,
#                                              ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
#                                              ["mean"])
# df1.to_csv("analysis/metric1_medium1k.csv")
# df1.head(10)

In [8]:
df1 = fr.get_embedding_similarity_metrics_per_dataset("medium1k", tags,
                                             ["minilm"],
                                             ["mean"])
df1.to_csv("analysis/metric1_medium1k_minilm.csv")
df1.head(10)

Calculating euclidean similarities: 100%|██████████| 20/20 [00:01<00:00, 11.82it/s]


,data_source,embedding_model,agg_method,metric_name,metric,between_all_nodes,between_shared_tags
0,medium1k,minilm,mean,cosine,mean,0.585432,0.607958
1,medium1k,minilm,mean,cosine,median,0.572935,0.596940
2,medium1k,minilm,mean,cosine,std_dev,0.070676,0.078426
3,medium1k,minilm,mean,soft_cosine,mean,0.577098,0.594107
4,medium1k,minilm,mean,soft_cosine,median,0.574291,0.591635
5,medium1k,minilm,mean,soft_cosine,std_dev,0.094922,0.102319
6,medium1k,minilm,mean,euclidean,mean,0.440976,0.462361
7,medium1k,minilm,mean,euclidean,median,0.425653,0.446588
8,medium1k,minilm,mean,euclidean,std_dev,0.066250,0.076233


In [9]:
df1 = fr.get_embedding_similarity_metrics_per_dataset("medium1k", tags,
                                             ["mpnet", "nomic", "bert", "specter", "word2vec"],
                                             ["mean"])
df1.to_csv("analysis/metric1_medium1k_rest.csv")
df1.head(10)

Calculating euclidean similarities: 100%|██████████| 20/20 [00:00<00:00, 26.06it/s]


,data_source,embedding_model,agg_method,metric_name,metric,between_all_nodes,between_shared_tags
0,medium1k,mpnet,mean,cosine,mean,0.567329,0.590647
1,medium1k,mpnet,mean,cosine,median,0.554505,0.578365
2,medium1k,mpnet,mean,cosine,std_dev,0.062230,0.071342
3,medium1k,mpnet,mean,soft_cosine,mean,0.553276,0.577064
4,medium1k,mpnet,mean,soft_cosine,median,0.549274,0.572625
5,medium1k,mpnet,mean,soft_cosine,std_dev,0.080918,0.088043
6,medium1k,mpnet,mean,euclidean,mean,0.424325,0.445684
7,medium1k,mpnet,mean,euclidean,median,0.410250,0.430301
8,medium1k,mpnet,mean,euclidean,std_dev,0.056616,0.067507
0,medium1k,nomic,mean,cosine,mean,0.777387,0.792314


### Cluster Purity, Homogeneity, and Completeness

In [12]:
k_values = [1, 2, 5, 10, 15, 20, 30, 40, 50, 65, 80, 100, 120, 150]

clustering_methods = {
    **{f"kmeans{k}": lambda x, k=k: clu.kmeans(x, k) for k in k_values},
    **{f"dbscan_eps{eps}_min{min_samples}": lambda x, eps=eps, min_samples=min_samples: clu.dbscan(x, eps=eps, min_samples=min_samples) for eps in [0.1, 0.3, 0.5, 0.7, 1.0] for min_samples in [3, 5, 10, 15]},
    **{f"gmm{n_components}": lambda x, n_components=n_components: clu.gmm(x, n_components) for n_components in k_values},
    **{f"birch{k}": lambda x, k=k: clu.birch(x, k) for k in k_values}
}

df2 = fr.compare_cluster_metrics("medium1k",
                                    ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
                                    ["mean"],
                                    clustering_methods,
                                    ids, tags, k=2)

df2.to_csv("analysis/metric2_medium1k.csv")
df2.head(10)

In [ ]:
df2 = fr.compare_cluster_metrics("medium1k",
                                    ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
                                    ["mean"],
                                    {
                                        "kmeans5": lambda x: clu.kmeans(x),
                                        "kmeans2": lambda x: clu.kmeans(x, 2),
                                        "kmeans10": lambda x: clu.kmeans(x, 10),
                                        "dbscan_default": lambda x: clu.dbscan(x, eps=0.),
                                        "dbscan_eps0.3": lambda x: clu.dbscan(x, eps=0.3),
                                        "dbscan_eps0.7": lambda x: clu.dbscan(x, eps=0.7),
                                        "gmm_default": lambda x: clu.gmm(x, n_components=3),
                                        "gmm_n4": lambda x: clu.gmm(x, n_components=4),
                                        "gmm_n5": lambda x: clu.gmm(x, n_components=5),
                                        "birch_default": lambda x: clu.birch(x),
                                        "birch_n5": lambda x: clu.birch(x, n_clusters=5),
                                        "birch_n10": lambda x: clu.birch(x, n_clusters=10)
                                    }, ids, tags, k=2)


### Edge Assignment Evaluation: Tag Connectivity and Degree of Separation

In [ ]:
k_values = [1, 2, 5, 10, 15, 20, 30, 40, 50]
threshold_values = [0.1, 0.3, 0.5, 0.7, 0.9]

edge_assignment_methods = {
    **{f"knn{k}": lambda sim_mat, ids, k=k: edge.knn(sim_mat, ids, k) for k in k_values},
    **{f"knn_mst{k}": lambda sim_mat, ids, k=k: edge.knn_mst(sim_mat, ids, k) for k in k_values},
    **{f"threshold{threshold}": lambda sim_mat, ids, threshold=threshold: edge.threshold_based_edge_assignment(sim_mat, ids, threshold) for threshold in threshold_values},
    **{f"mutual_knn{k}": lambda sim_mat, ids, k=k: edge.mutual_knn_edge_assignment(sim_mat, ids, k) for k in k_values},
    **{f"spectral{n_clusters}": lambda sim_mat, ids, n_clusters=n_clusters: edge.spectral_clustering_edge_assignment(sim_mat, ids, n_clusters) for n_clusters in k_values}
}

df3 = fr.compare_edge_assignment_metrics("interview_prep",
                                        ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
                                        ["mean"],
                                        {
                                            # "kmeans5": lambda x: clu.kmeans(x), # pick the best form above... to save time
                                            # "kmeans2": lambda x: clu.kmeans(x, 2)
                                        },
                                        edge_assignment_methods,
                                        ids, tags, titles, max_depth=3)

df3.to_csv("analysis/metric3_medium1k.csv")
print(df3.head(10))

In [ ]:
# df3 = fr.compare_edge_assignment_metrics("interview_prep",
#                                         ["minilm", "mpnet", "nomic", "bert", "specter", "word2vec"],
#                                         ["mean"],
#                                         {
#                                             "kmeans5": lambda x: clu.kmeans(x),
#                                             "kmeans2": lambda x: clu.kmeans(x, 2)
#                                         },
#                                         {
#                                             "knn3": lambda sim_mat, ids: edge.knn(sim_mat, ids, k=3),
#                                             "knn5": lambda sim_mat, ids: edge.knn(sim_mat, ids, k=5)
#                                         }, ids, tags, titles, max_depth=3)
# df3.to_csv("analysis/metric3_medium1k.csv")
# df3.head(10)